In [1]:
%%time
import datetime as dt
import sys
import os

import scipy as sp
import numpy as np
import pandas as pd

from IPython.display import display, HTML

import fastparquet

import matplotlib.pyplot as plt
%matplotlib inline

CPU times: user 787 ms, sys: 141 ms, total: 928 ms
Wall time: 2.37 s


In [6]:
%%time
cols_fhv = ['dispatching_base_num', 'pickup_date', 'locationid']
cols_fhv_new = {'dispatching_base_num': 'disp_num', 'pickup_date': 'pickup_date', 'locationid': 'loc_id'}


cols_grn_nn = {'vendorid': 'v_id', 'lpep_pickup_datetime': 'pick_time', 'lpep_dropoff_datetime': 'drop_time',
               'ratecodeid': 'rate_code', 'pickup_longitude': 'pick_lng', 'pickup_latitude': 'pick_lat',
               'dropoff_longitude': 'drop_lng', 'dropoff_latitude': 'drop_lat', 'passenger_count': 'count',
               'trip_distance': 'distance', 'fare_amount': 'fare', 'total_amount': 'total', 
               'payment_type': 'pay_type'}

cols_grn_new = {'vendorID': 'v_id', 'lpep_pickup_datetime': 'pick_time', 
                'lpep_dropoff_datetime': 'drop_time', 'store_and_fwd_flag': 'flag', 
                'ratecodeid': 'rate_id', 'pickup_longitude': 'pick_lngtd', 
                'pickup_latitude': 'pick_lattd', 'dropoff_longitude': 'drop_lngtd',
                'dropoff_latitude': 'drop_lattd', 'passenger_count': 'psgr_ct', 
                'trip_distance': 'dist*100', 'fare_amount': 'fare*100', 'extra': 'extra*100',
                'mta_tax': 'mta_tax*100', 'tip_amount': 'tip*100', 'tolls_amount': 'toll*100',
                'ehail_fee': 'ehail*100', 'improvement_surcharge': 'improvement*100', 
                'total_amount': 'total*100', 'payment_type': 'payment_type', 'trip_type': 'trip_type'}


cols_ylw_nn = {'vendorid': 'v_id', 'tpep_pickup_datetime': 'pick_time', 'tpep_dropoff_datetime': 'drop_time',
               'passenger_count': 'count', 'trip_distance': 'distance', 'pickup_longitude': 'pick_lng',
               'pickup_latitude': 'pick_lat', 'ratecodeid': 'rate_code', 'dropoff_longitude': 'drop_lng',
               'dropoff_latitude': 'drop_lat', 'payment_type': 'pay_type', 'fare_amount': 'fare',
               'total_amount': 'total'}

cols_ylw_new = {'vendorid': 'v_id', 'tpep_pickup_datetime': 'pick_time', 
                'tpep_dropoff_datetime': 'drop_time', 'store_and_fwd_flag': 'flag', 
                'ratecodeid': 'rate_id', 'pickup_longitude': 'pick_lngtd', 
                'pickup_latitude': 'pick_lattd', 'dropoff_longitude': 'drop_lngtd',
                'dropoff_latitude': 'drop_lattd', 'passenger_count': 'psgr_ct', 
                'trip_distance': 'dist*100', 'fare_amount': 'fare*100', 'extra': 'extra*100',
                'mta_tax': 'mta_tax*100', 'tip_amount': 'tip*100', 'tolls_amount': 'toll*100',
                'improvement_surcharge': 'improvement*100', 
                'total_amount': 'total*100', 'payment_type': 'payment_type'}

cols_grn_dt = ['lpep_pickup_datetime', 'lpep_dropoff_datetime']
cols_ylw_dt = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']
cols_grn_mul = ['trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 
                'ehail_fee', 'improvement_surcharge', 'total_amount']
cols_ylw_mul = ['trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 
                'improvement_surcharge', 'total_amount']
cols_grn_cat = ['vendorid', 'store_and_fwd_flag', 'ratecodeid', 'passenger_count', 'trip_distance', 
                'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee', 
                'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type']
cols_ylw_cat = ['vendorid', 'store_and_fwd_flag', 'ratecodeid', 'passenger_count', 'trip_distance', 
                'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
                'total_amount', 'payment_type']
cols_grn_cat_num = ['passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 
                    'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'total_amount']
cols_ylw_cat_num = ['passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 
                    'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']


CPU times: user 18 µs, sys: 1e+03 ns, total: 19 µs
Wall time: 26 µs


In [ ]:
# Important parameters for green:
    # vendorid -> int8
    # pickup_time -> dt
    # dropoff_time -> dt
    # count -> int8
    # distance -> float16
    # pickup_loc -> geopandas_point
    # dropoff_loc -> geopandas_point
    # ratecode -> int8
    # pay_type -> int8
    # fare -> float16
    # total -> float16
    # trip_type -> int8
    

# Important parameters for yellow:
    # vendorid -> int8
    # pickup_time -> dt
    # dropoff_time -> dt
    # count -> int8
    # distance -> float16
    # pickup_loc -> geopandas_point
    # dropoff_loc -> geopandas_point
    # ratecode -> int8
    # pay_type -> int8
    # fare -> float16
    # total -> float16


In [2]:
%%time
year0 = ['2016-06', '2016-05', '2016-04', '2016-03', '2016-02', '2016-01', '2015-12', '2015-11', '2015-10',
         '2015-09', '2015-08', '2015-07', '2015-06', '2015-05', '2015-04', '2015-03', '2015-02', '2015-01']
year_month = year0[0]
#file_fhv = 'fhv_tripdata_' + year_month
file_grn = 'green_tripdata_' + year_month
file_ylw = 'yellow_tripdata_' + year_month
filepath = 'nyc_gov_data'
#fileaddr_fhv = os.path.join(filepath,file_fhv+'.csv')
fileaddr_grn = os.path.join(filepath,file_grn+'.csv')
fileaddr_ylw = os.path.join(filepath,file_ylw+'.csv')
#print('fhv')
#df_fhv = pd.read_csv(fileaddr_fhv)
print('green')
df_grn = pd.read_csv(fileaddr_grn)
print('yellow')
df_ylw = pd.read_csv(fileaddr_ylw)
#df_fhv.columns = df_fhv.columns.str.strip()
df_grn.columns = df_grn.columns.str.strip()
df_ylw.columns = df_ylw.columns.str.strip()
#df_fhv.columns = df_fhv.columns.str.lower()
df_grn.columns = df_grn.columns.str.lower()
df_ylw.columns = df_ylw.columns.str.lower()
#df_fhv.fillna(0, inplace=True)
#df_grn.fillna(0, inplace=True)
#df_ylw.fillna(0, inplace=True)


#print('fhv')
#print(df_fhv.dtypes)
#display(df_fhv.head())
print(year_month)
print('green')
print(df_grn.dtypes)
display(df_grn.head())
print('yellow')
print(df_ylw.dtypes)
display(df_ylw.head())

# 2016-01: 
#   CPU times: user 1min 52s, sys: 13 s, total: 2min 5s
#   Wall time: 4min 37s
# 2016-02: 
# .  CPU times: user 1min 57s, sys: 13.4 s, total: 2min 10s
# .  Wall time: 2min 50s
# 2016-03:
# .  CPU times: user 2min 3s, sys: 14.3 s, total: 2min 17s
# .  Wall time: 3min 24s
# 2016-04:
# .  CPU times: user 2min 5s, sys: 16.2 s, total: 2min 22s
# .  Wall time: 3min 56s
# 2016-05:
# .  CPU times: user 2min 11s, sys: 21.6 s, total: 2min 33s
# .  Wall time: 6min 16s
# 2016-06:
# .  CPU times: user 1min 58s, sys: 12.4 s, total: 2min 10s
# .  Wall time: 2min 12s


green
yellow
2016-06
green
vendorid                   int64
lpep_pickup_datetime      object
lpep_dropoff_datetime     object
store_and_fwd_flag        object
ratecodeid                 int64
pickup_longitude         float64
pickup_latitude          float64
dropoff_longitude        float64
dropoff_latitude         float64
passenger_count            int64
trip_distance            float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
ehail_fee                float64
improvement_surcharge    float64
total_amount             float64
payment_type               int64
trip_type                float64
dtype: object


,vendorid,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,ratecodeid,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type
0,2,2016-06-01 02:46:38,2016-06-01 03:06:40,N,1,-73.930580,40.695179,-74.000053,40.729046,1,...,19.5,0.5,0.5,6.24,0.0,NaN,0.3,27.04,1,1.0
1,2,2016-06-01 02:55:26,2016-06-01 03:06:52,N,1,-73.946930,40.792553,-73.951569,40.825161,1,...,11.5,0.5,0.5,2.56,0.0,NaN,0.3,15.36,1,1.0
2,2,2016-06-01 02:50:36,2016-06-01 03:08:39,N,1,-73.944534,40.823956,-73.994659,40.750423,1,...,23.5,0.5,0.5,2.00,0.0,NaN,0.3,26.80,1,1.0
3,2,2016-06-01 02:57:04,2016-06-01 03:07:52,N,1,-73.952209,40.823872,-73.914360,40.814697,1,...,10.5,0.5,0.5,0.00,0.0,NaN,0.3,11.80,2,1.0
4,2,2016-06-01 02:52:03,2016-06-01 03:08:12,N,1,-73.957977,40.717827,-73.954018,40.655121,3,...,16.5,0.5,0.5,0.00,0.0,NaN,0.3,17.80,1,1.0


yellow
vendorid                   int64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count            int64
trip_distance            float64
pickup_longitude         float64
pickup_latitude          float64
ratecodeid                 int64
store_and_fwd_flag        object
dropoff_longitude        float64
dropoff_latitude         float64
payment_type               int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
dtype: object


,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,ratecodeid,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-06-09 21:06:36,2016-06-09 21:13:08,2,0.79,-73.983360,40.760937,1,N,-73.977463,40.753979,2,6.0,0.5,0.5,0.00,0.0,0.3,7.30
1,2,2016-06-09 21:06:36,2016-06-09 21:35:11,1,5.22,-73.981720,40.736668,1,N,-73.981636,40.670242,1,22.0,0.5,0.5,4.00,0.0,0.3,27.30
2,2,2016-06-09 21:06:36,2016-06-09 21:13:10,1,1.26,-73.994316,40.751072,1,N,-74.004234,40.742168,1,6.5,0.5,0.5,1.56,0.0,0.3,9.36
3,2,2016-06-09 21:06:36,2016-06-09 21:36:10,1,7.39,-73.982361,40.773891,1,N,-73.929466,40.851540,1,26.0,0.5,0.5,1.00,0.0,0.3,28.30
4,2,2016-06-09 21:06:36,2016-06-09 21:23:23,1,3.10,-73.987106,40.733173,1,N,-73.985909,40.766445,1,13.5,0.5,0.5,2.96,0.0,0.3,17.76


CPU times: user 1min 32s, sys: 12.6 s, total: 1min 45s
Wall time: 3min 48s


In [33]:
%%time
#col_grn = df_grn.columns.tolist()
col_grn2 = ['v_id', 'pickup_datetime', 'dropoff_datetime', 'flag', 'rate_code', 'pickup_lng',
            'pickup_lat', 'dropoff_lng', 'dropoff_lat', 'psgr_count', 'distance', 'fare', 'extra',
            'mta_tax', 'tip', 'tolls', 'ehail_fee', 'improvement_surcharge', 'total', 'pay_type',
            'trip_type']
#df_grn.columns = col_grn2
#col_ylw = df_ylw.columns.tolist()
col_ylw2 = ['v_id', 'pickup_datetime', 'dropoff_datetime', 'psgr_count', 'distance', 'pickup_lng',
            'pickup_lat', 'rate_code', 'flag', 'dropoff_lng', 'dropoff_lat', 'pay_type', 'fare',
            'extra', 'mta_tax', 'tip', 'tolls', 'improvement_surcharge', 'total']
#df_ylw.columns = col_ylw2

display(np.finfo(np.float16))

#df_grn.loc[:,'dummy_col1'] = df_grn.iloc[:,0][0]
#df_grn.loc[:,'dummy_col2'] = df_grn.iloc[:,0][1]
#display(df_grn.head())
#df_grn.rename(index=str, columns=cols_grn_new, inplace=True)
#print(col_ylw)

finfo(resolution=0.001, min=-6.55040e+04, max=6.55040e+04, dtype=float16)

CPU times: user 2.85 ms, sys: 1.72 ms, total: 4.58 ms
Wall time: 6.74 ms


In [34]:
%%time
col_dt = ['pickup_datetime', 'dropoff_datetime']
col_grn_cat = ['v_id', 'flag', 'rate_code', 'pay_type', 'trip_type']
col_grn_int8 = ['psgr_count']
col_grn_flt16 = ['distance', 'fare', 'extra', 'mta_tax', 'tip', 'tolls', 'ehail_fee',
                  'improvement_surcharge', 'total']
col_green = ['v_id', 'pickup_datetime', 'dropoff_datetime', 'rate_code', 'pickup_lng', 'pickup_lat',
             'dropoff_lng', 'dropoff_lat', 'psgr_count', 'distance', 'fare', 'total', 'pay_type', 'trip_type']


col_ylw_cat = ['v_id', 'rate_code', 'flag', 'pay_type']
col_ylw_int8 = ['psgr_count']
col_ylw_flt16 = ['distance', 'fare', 'extra', 'mta_tax', 'tip', 'tolls', 'improvement_surcharge', 'total']
col_yellow = ['v_id', 'pickup_datetime', 'dropoff_datetime', 'psgr_count', 'distance', 'pickup_lng',
            'pickup_lat', 'rate_code', 'dropoff_lng', 'dropoff_lat', 'pay_type', 'fare', 'total']


for c in col_dt:
    df_grn.loc[:,c] = pd.to_datetime(df_grn.loc[:,c],format='%Y-%m-%d %H:%M:%S')
    df_ylw.loc[:,c] = pd.to_datetime(df_grn.loc[:,c],format='%Y-%m-%d %H:%M:%S')
    
df_grn.loc[:,'psgr_count'] = df_grn.loc[:,'psgr_count'].astype(np.int8)
df_ylw.loc[:,'psgr_count'] = df_ylw.loc[:,'psgr_count'].astype(np.int8)

for c in col_grn_cat:
    df_grn.loc[:,c] = df_grn.loc[:,c].astype('category')
for c in col_ylw_cat:
    df_ylw.loc[:,c] = df_ylw.loc[:,c].astype('category')
for c in col_grn_flt16:
    df_grn.loc[:,c] = df_grn.loc[:,c].astype(np.float16)
for c in col_ylw_flt16:
    df_ylw.loc[:,c] = df_ylw.loc[:,c].astype(np.float16)

    
display(df_grn.loc[:5, col_green])
display(df_ylw.loc[:5, col_yellow])

,v_id,pickup_datetime,dropoff_datetime,rate_code,pickup_lng,pickup_lat,dropoff_lng,dropoff_lat,psgr_count,distance,fare,total,pay_type,trip_type
0,2,2016-06-01 02:46:38,2016-06-01 03:06:40,1,-73.930580,40.695179,-74.000053,40.729046,1,5.238281,19.5,27.046875,1,1.0
1,2,2016-06-01 02:55:26,2016-06-01 03:06:52,1,-73.946930,40.792553,-73.951569,40.825161,1,3.140625,11.5,15.359375,1,1.0
2,2,2016-06-01 02:50:36,2016-06-01 03:08:39,1,-73.944534,40.823956,-73.994659,40.750423,1,7.500000,23.5,26.796875,1,1.0
3,2,2016-06-01 02:57:04,2016-06-01 03:07:52,1,-73.952209,40.823872,-73.914360,40.814697,1,2.269531,10.5,11.796875,2,1.0
4,2,2016-06-01 02:52:03,2016-06-01 03:08:12,1,-73.957977,40.717827,-73.954018,40.655121,3,4.898438,16.5,17.796875,1,1.0
5,2,2016-06-01 02:59:03,2016-06-01 03:09:25,1,-73.965324,40.711033,-73.989685,40.714169,1,2.759766,11.0,14.757812,1,1.0


,v_id,pickup_datetime,dropoff_datetime,psgr_count,distance,pickup_lng,pickup_lat,rate_code,dropoff_lng,dropoff_lat,pay_type,fare,total
0,2,2016-06-01 02:46:38,2016-06-01 03:06:40,2,0.790039,-73.983360,40.760937,1,-73.977463,40.753979,2,6.0,7.300781
1,2,2016-06-01 02:55:26,2016-06-01 03:06:52,1,5.218750,-73.981720,40.736668,1,-73.981636,40.670242,1,22.0,27.296875
2,2,2016-06-01 02:50:36,2016-06-01 03:08:39,1,1.259766,-73.994316,40.751072,1,-74.004234,40.742168,1,6.5,9.359375
3,2,2016-06-01 02:57:04,2016-06-01 03:07:52,1,7.390625,-73.982361,40.773891,1,-73.929466,40.851540,1,26.0,28.296875
4,2,2016-06-01 02:52:03,2016-06-01 03:08:12,1,3.099609,-73.987106,40.733173,1,-73.985909,40.766445,1,13.5,17.765625
5,2,2016-06-01 02:59:03,2016-06-01 03:09:25,1,2.169922,-73.995201,40.739491,1,-73.993202,40.762642,1,10.5,14.156250


CPU times: user 25.5 s, sys: 15.6 s, total: 41 s
Wall time: 39 s


In [35]:
%%time
fastparquet.write(file_grn+'00.parq', df_grn, compression='GZIP')
print(file_grn+' done.')
fastparquet.write(file_ylw+'00.parq', df_ylw, compression='GZIP')
print(file_ylw+' done.')


green_tripdata_2016-06 done.
yellow_tripdata_2016-06 done.
CPU times: user 1min 48s, sys: 3.83 s, total: 1min 52s
Wall time: 2min 31s


In [4]:
%%time
col_dt = ['pickup_datetime', 'dropoff_datetime']
col_grn2_cat = ['v_id', 'flag', 'rate_code', 'pay_type', 'trip_type']
col_grn2_int8 = ['psgr_count']
col_grn2_flt16 = ['distance', 'fare', 'extra', 'mta_tax', 'tip', 'tolls', 'ehail_fee',
                  'improvement_surcharge', 'total']

col_ylw_cat = ['v_id', 'rate_code', 'flag', 'pay_type']
col_ylw2_int8 = ['psgr_count']
col_ylw2_flt16 = ['distance', 'fare', 'extra', 'mta_tax', 'tip', 'tolls', 'improvement_surcharge', 'total']

for c in col_grn_dt:
    df_grn.loc[:,c] = pd.to_datetime(df_grn.loc[:,c],format='%Y-%m-%d %H:%M:%S')
    df_ylw.loc[:,c] = pd.to_datetime(df_grn.loc[:,c],format='%Y-%m-%d %H:%M:%S')
    
df_grn.loc[:,'psgr_count'] = df_grn.loc[:,'psgr_count'].astype(np.int8)
df_ylw.loc[:,'psgr_count'] = df_ylw.loc[:,'psgr_count'].astype(np.int8)


    

df_fhv.loc[:,'pickup_date'] = pd.to_datetime(df_fhv.loc[:,'pickup_date'], format='%Y-%m-%d %H:%M:%S')
df_fhv.loc[:,'locationid'] = df_fhv.loc[:,'locationid'].astype('category')
df_fhv.loc[:,'dispatching_base_num'] = df_fhv.loc[:,'dispatching_base_num'].astype('category')

for c in cols_grn_mul:
    df_grn.loc[:,c] = df_grn.loc[:,c]*100
for c in cols_ylw_mul:
    df_ylw.loc[:,c] = df_ylw.loc[:,c]*100
for c in cols_grn_dt:
    df_grn.loc[:,c] = pd.to_datetime(df_grn.loc[:,c],format='%Y-%m-%d %H:%M:%S')
for c in cols_ylw_dt:
    df_ylw.loc[:,c] = pd.to_datetime(df_ylw.loc[:,c],format='%Y-%m-%d %H:%M:%S')
for c in cols_grn_cat:
    df_grn.loc[:,c] = df_grn.loc[:,c].astype('category')
for c in cols_ylw_cat:
    df_ylw.loc[:,c] = df_ylw.loc[:,c].astype('category')
for c in cols_grn_cat_num:
    df_grn.loc[:,c] = df_grn.loc[:,c].astype(int)
for c in cols_ylw_cat_num:
    df_ylw.loc[:,c] = df_ylw.loc[:,c].astype(int)

#df_fhv.rename(index=str, columns=cols_fhv_new, inplace=True)
#df_grn.rename(index=str, columns=cols_grn_new, inplace=True)
#df_ylw.rename(index=str, columns=cols_ylw_new, inplace=True)


#print('fhv')
#print(df_fhv.dtypes)
#display(df_fhv.head())
print('green')
print(df_grn.dtypes)
display(df_grn.head())
print('yellow')
print(df_ylw.dtypes)
display(df_ylw.head())

# 2016-01:
# .  CPU times: user 1min 6s, sys: 24.9 s, total: 1min 31s
# .  Wall time: 1min 31s
# 2016-02:
# .  CPU times: user 1min 5s, sys: 25.2 s, total: 1min 30s
# .  Wall time: 1min 48s
# 2016-03:
# .  CPU times: user 1min 10s, sys: 26.9 s, total: 1min 37s
# .  Wall time: 1min 59s
# 2016-04:
# .  CPU times: user 1min 14s, sys: 26.5 s, total: 1min 41s
# .  Wall time: 1min 27s
# 2016-05:
# .  CPU times: user 1min 5s, sys: 32.1 s, total: 1min 37s
# .  Wall time: 4min 26s
# 2016-06:
# .  CPU times: user 1min 16s, sys: 25.1 s, total: 1min 41s
# .  Wall time: 1min 16s


fhv
dispatching_base_num          category
pickup_date             datetime64[ns]
locationid                    category
dtype: object


,dispatching_base_num,pickup_date,locationid
0,B00001,2015-12-01 04:30:00,0.0
1,B00001,2015-12-01 04:30:00,0.0
2,B00001,2015-12-01 04:45:00,0.0
3,B00001,2015-12-01 05:15:00,0.0
4,B00001,2015-12-01 05:20:00,0.0


green
vendorid                       category
lpep_pickup_datetime     datetime64[ns]
lpep_dropoff_datetime    datetime64[ns]
store_and_fwd_flag             category
ratecodeid                     category
pickup_longitude                float64
pickup_latitude                 float64
dropoff_longitude               float64
dropoff_latitude                float64
passenger_count                   int64
trip_distance                     int64
fare_amount                       int64
extra                             int64
mta_tax                           int64
tip_amount                        int64
tolls_amount                      int64
ehail_fee                         int64
improvement_surcharge             int64
total_amount                      int64
payment_type                   category
trip_type                      category
dtype: object


,vendorid,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,ratecodeid,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type
0,2,2015-12-01 00:12:54,2015-12-01 00:18:18,N,1,-73.844681,40.721508,-73.836334,40.708878,1,...,650,50,50,156,0,0,30,936,1,1.0
1,2,2015-12-01 00:48:19,2015-12-01 00:59:31,N,1,-73.807030,40.699657,-73.863670,40.691143,1,...,1250,50,50,200,0,0,30,1580,1,1.0
2,2,2015-12-01 00:06:13,2015-12-01 00:20:40,N,1,-73.961815,40.805641,-73.925980,40.824123,2,...,1350,50,50,444,0,0,30,1923,1,1.0
3,2,2015-12-01 00:43:38,2015-12-01 00:59:37,N,1,-73.945221,40.808384,-73.959587,40.801357,1,...,1250,50,50,276,0,0,30,1655,1,1.0
4,2,2015-12-01 00:04:50,2015-12-01 00:09:40,N,1,-73.939018,40.805542,-73.943977,40.813740,5,...,550,50,50,100,0,0,30,780,1,1.0


yellow
vendorid                       category
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int64
trip_distance                     int64
pickup_longitude                float64
pickup_latitude                 float64
ratecodeid                     category
store_and_fwd_flag             category
dropoff_longitude               float64
dropoff_latitude                float64
payment_type                   category
fare_amount                       int64
extra                             int64
mta_tax                           int64
tip_amount                        int64
tolls_amount                      int64
improvement_surcharge             int64
total_amount                      int64
dtype: object


,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,ratecodeid,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-12-01 00:00:00,2015-12-01 00:05:16,5,96,-73.979942,40.765381,1,N,-73.966309,40.763088,1,550,50,50,100,0,30,780
1,2,2015-12-01 00:00:00,2015-12-01 00:00:00,2,269,-73.972336,40.762379,1,N,-73.993629,40.745998,1,2150,0,50,334,0,30,2564
2,2,2015-12-01 00:00:00,2015-12-01 00:00:00,1,262,-73.968849,40.764530,1,N,-73.974548,40.791641,1,1700,0,50,356,0,30,2136
3,1,2015-12-01 00:00:01,2015-12-01 00:05:56,1,120,-73.993935,40.741684,1,N,-73.997665,40.747467,1,650,50,50,20,0,30,800
4,1,2015-12-01 00:00:01,2015-12-01 00:09:28,2,300,-73.988922,40.726990,1,N,-73.975594,40.696869,2,1100,50,50,0,0,30,1230


CPU times: user 1min 20s, sys: 25.7 s, total: 1min 46s
Wall time: 1min 15s


In [5]:
%%time
fastparquet.write(file_fhv+'.parq', df_fhv, compression='GZIP')
print(file_fhv+' done.')
fastparquet.write(file_grn+'.parq', df_grn, compression='GZIP')
print(file_grn+' done.')
fastparquet.write(file_ylw+'.parq', df_ylw, compression='GZIP')
print(file_ylw+' done.')

# 2016-01:
# .  CPU times: user 1min 51s, sys: 5.62 s, total: 1min 57s
# .  Wall time: 2min 56s
# 2016-02:
# .  CPU times: user 1min 56s, sys: 6.01 s, total: 2min 2s
# .  Wall time: 2min 55s
# 2016-03:
# .  CPU times: user 2min 4s, sys: 6.25 s, total: 2min 10s
# .  Wall time: 3min 8s
# 2016-04:
# .  CPU times: user 2min 3s, sys: 8.11 s, total: 2min 11s
# .  Wall time: 3min 31s
# 2016-05:
# .  CPU times: user 2min 13s, sys: 9.02 s, total: 2min 22s
# .  Wall time: 6min 0s
# 2016-06:
# .  CPU times: user 1min 56s, sys: 4.72 s, total: 2min
# .  Wall time: 2min 8s


fhv_tripdata_2015-12 done.
green_tripdata_2015-12 done.
yellow_tripdata_2015-12 done.
CPU times: user 1min 57s, sys: 4.81 s, total: 2min 1s
Wall time: 2min 1s


In [21]:
#%%timeit
#df_grn.rename(columns = col_grn_dict)
col_grn2 = ['v_id', 'pickup_datetime', 'dropoff_datetime', 'flag', 'rate_code', 'pickup_lng',
            'pickup_lat', 'dropoff_lng', 'dropoff_lat', 'psgr_count', 'distance', 'fare', 'extra',
            'mta_tax', 'tip', 'tolls', 'ehail_fee', 'improvement_surcharge', 'total', 'pay_type',
            'trip_type']

display(df_grn.head(10))
df_grn.columns = col_grn2
display(df_grn.head(10))

,vendorid,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,ratecodeid,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type
0,2,2016-06-01 02:46:38,2016-06-01 03:06:40,N,1,-73.930580,40.695179,-74.000053,40.729046,1,...,19.5,0.5,0.5,6.24,0.0,NaN,0.3,27.04,1,1.0
1,2,2016-06-01 02:55:26,2016-06-01 03:06:52,N,1,-73.946930,40.792553,-73.951569,40.825161,1,...,11.5,0.5,0.5,2.56,0.0,NaN,0.3,15.36,1,1.0
2,2,2016-06-01 02:50:36,2016-06-01 03:08:39,N,1,-73.944534,40.823956,-73.994659,40.750423,1,...,23.5,0.5,0.5,2.00,0.0,NaN,0.3,26.80,1,1.0
3,2,2016-06-01 02:57:04,2016-06-01 03:07:52,N,1,-73.952209,40.823872,-73.914360,40.814697,1,...,10.5,0.5,0.5,0.00,0.0,NaN,0.3,11.80,2,1.0
4,2,2016-06-01 02:52:03,2016-06-01 03:08:12,N,1,-73.957977,40.717827,-73.954018,40.655121,3,...,16.5,0.5,0.5,0.00,0.0,NaN,0.3,17.80,1,1.0
5,2,2016-06-01 02:59:03,2016-06-01 03:09:25,N,1,-73.965324,40.711033,-73.989685,40.714169,1,...,11.0,0.5,0.5,2.46,0.0,NaN,0.3,14.76,1,1.0
6,2,2016-06-01 02:58:43,2016-06-01 03:03:00,N,1,-73.921333,40.766644,-73.925423,40.761703,1,...,5.5,0.5,0.5,0.00,0.0,NaN,0.3,6.80,2,1.0
7,2,2016-06-01 02:37:36,2016-06-02 00:00:00,N,1,-73.949203,40.682667,-73.983582,40.725903,1,...,25.5,0.5,0.5,0.00,0.0,NaN,0.3,26.80,2,1.0
8,2,2016-06-01 02:56:23,2016-06-01 03:08:51,N,1,-73.951576,40.713966,-73.918785,40.689823,1,...,11.5,0.5,0.5,0.00,0.0,NaN,0.3,12.80,2,1.0
9,2,2016-06-01 02:52:21,2016-06-01 03:11:12,N,1,-73.904305,40.745346,-73.901741,40.745640,1,...,16.0,0.5,0.5,0.00,0.0,NaN,0.3,17.30,2,1.0


,v_id,pickup_datetime,dropoff_datetime,flag,rate_code,pickup_lng,pickup_lat,dropoff_lng,dropoff_lat,psgr_count,...,fare,extra,mta_tax,tip,tolls,ehail_fee,improvement_surcharge,total,pay_type,trip_type
0,2,2016-06-01 02:46:38,2016-06-01 03:06:40,N,1,-73.930580,40.695179,-74.000053,40.729046,1,...,19.5,0.5,0.5,6.24,0.0,NaN,0.3,27.04,1,1.0
1,2,2016-06-01 02:55:26,2016-06-01 03:06:52,N,1,-73.946930,40.792553,-73.951569,40.825161,1,...,11.5,0.5,0.5,2.56,0.0,NaN,0.3,15.36,1,1.0
2,2,2016-06-01 02:50:36,2016-06-01 03:08:39,N,1,-73.944534,40.823956,-73.994659,40.750423,1,...,23.5,0.5,0.5,2.00,0.0,NaN,0.3,26.80,1,1.0
3,2,2016-06-01 02:57:04,2016-06-01 03:07:52,N,1,-73.952209,40.823872,-73.914360,40.814697,1,...,10.5,0.5,0.5,0.00,0.0,NaN,0.3,11.80,2,1.0
4,2,2016-06-01 02:52:03,2016-06-01 03:08:12,N,1,-73.957977,40.717827,-73.954018,40.655121,3,...,16.5,0.5,0.5,0.00,0.0,NaN,0.3,17.80,1,1.0
5,2,2016-06-01 02:59:03,2016-06-01 03:09:25,N,1,-73.965324,40.711033,-73.989685,40.714169,1,...,11.0,0.5,0.5,2.46,0.0,NaN,0.3,14.76,1,1.0
6,2,2016-06-01 02:58:43,2016-06-01 03:03:00,N,1,-73.921333,40.766644,-73.925423,40.761703,1,...,5.5,0.5,0.5,0.00,0.0,NaN,0.3,6.80,2,1.0
7,2,2016-06-01 02:37:36,2016-06-02 00:00:00,N,1,-73.949203,40.682667,-73.983582,40.725903,1,...,25.5,0.5,0.5,0.00,0.0,NaN,0.3,26.80,2,1.0
8,2,2016-06-01 02:56:23,2016-06-01 03:08:51,N,1,-73.951576,40.713966,-73.918785,40.689823,1,...,11.5,0.5,0.5,0.00,0.0,NaN,0.3,12.80,2,1.0
9,2,2016-06-01 02:52:21,2016-06-01 03:11:12,N,1,-73.904305,40.745346,-73.901741,40.745640,1,...,16.0,0.5,0.5,0.00,0.0,NaN,0.3,17.30,2,1.0
